In [10]:
import json
from bs4 import BeautifulSoup
from collections import defaultdict

with open("publications.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

publications = raw_data.get("publications", [])

grouped = defaultdict(list)
for pub in publications:
    try:
        year = int(pub.get("year", 0))
    except (ValueError, TypeError):
        year = "Unknown"
    grouped[year].append(pub)

soup = BeautifulSoup("<html><head><title>Publications</title></head><body></body></html>", "html.parser")

pub_head = soup.new_tag("div", id="pubHead")

page_top_anchor = soup.new_tag("a", attrs={"name": "page-top"})
pub_head.append(page_top_anchor)

font_outer = soup.new_tag("font", size="2")
bold_title = soup.new_tag("b")
bold_title.string = "Shea Group Publications"
font_outer.append(bold_title)
font_outer.append(" [ ")

font_inner = soup.new_tag("font", size="2")

group_home = soup.new_tag("a", href="index.html")
group_home.string = "Group Home"
font_inner.append(group_home)
font_inner.append(" / ")

years_sorted = sorted(
    [y for y in grouped.keys() if isinstance(y, int)], reverse=True
) + [y for y in grouped.keys() if not isinstance(y, int)]

for i, year in enumerate(years_sorted):
    year_link = soup.new_tag("a", href=f"#pub{year}")
    year_link.string = str(year)
    font_inner.append(year_link)
    if i < len(years_sorted) - 1:
        font_inner.append(" / ")

font_outer.append(font_inner)
font_outer.append(" ]")
pub_head.append(font_outer)
soup.body.append(pub_head)

container = soup.new_tag("div", id="publications-container")
soup.body.append(container)

for year in years_sorted:
    year_div = soup.new_tag("div", **{"class": "year-group"})

    year_anchor = soup.new_tag("a", attrs={"name": f"pub{year}"})
    year_div.append(year_anchor)

    h2 = soup.new_tag("h2")
    h2.string = str(year)
    year_div.append(h2)

    form = soup.new_tag("form")
    input_button = soup.new_tag("input", type="BUTTON", value="Back to top of page")
    input_button["onclick"] = "window.location.href='#page-top'"
    form.append(input_button)
    year_div.append(form)

    for pub in grouped[year]:
        pub_div = soup.new_tag("div", **{"class": "publication"})

        title = soup.new_tag("h3")
        title.string = pub.get("title", "No Title")
        pub_div.append(title)

        authors = soup.new_tag("p")
        authors.string = f"Authors: {pub.get('authors', 'Unknown')}"
        pub_div.append(authors)

        journal = soup.new_tag("p")
        journal.string = f"Journal: {pub.get('source', 'Unknown')}"
        pub_div.append(journal)

        year_p = soup.new_tag("p")
        year_p.string = f"Year: {pub.get('year', 'Unknown')}"
        pub_div.append(year_p)

        if pub.get("doi"):
            doi_p = soup.new_tag("p")
            doi_p.string = "DOI: "
            doi_link = soup.new_tag("a", href=f"https://doi.org/{pub['doi']}", **{"class": "doi", "target": "_blank"})
            doi_link.string = pub['doi']
            doi_p.append(doi_link)
            pub_div.append(doi_p)

        year_div.append(pub_div)

    container.append(year_div)

with open("updated_publications3.html", "w", encoding="utf-8") as f:
    f.write(soup.prettify())

print("✅ NEW HTML generated ;)")


✅ NEW HTML generated ;)
